#### Coupled Ordinary DE's and Orbital Mechanics

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import numpy as np

In [3]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccentricity):
        self.x = np.array(2) #x and y position
        self.v = np.array(2) #x and y velocity
        self.a_g = np.array(2) #x and y acceleration
        self.t = 0.0 #current time
        self.dt = 0.0 #current timestep
        self.a = semimajor #semimajor axis of the orbit
        self.e = eccentricity #eccentricity of the orbit
        self.istep = 0 #current integrer 

In [4]:
solar_system = {"M_sun":1.0, "G":39.4784176043574320}

In [5]:
def solar_circular_velocity(p,solar_system):
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2+p.x[1]**2)**0.5
    
    return (G*M/r)**0.5

In [7]:
def solar_gravitational_acceleration(p, solar_system):
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2+p.x[1]**2)**0.5
    
    #acceleration in AU/yr/yr
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
    else:
        theta = np.atan(p.x[1], p.x[0])
    
    #set the x and y components of the velocity
    p.a_g[0] = a_grav * np.cos(theta)
    p.a_g[1] = a_grav * np.sin(theta)

In [8]:
def calc_dt(p):
    ETA_TIME_STEP = 0.0004
    eta = ETA_TIME_STEP
    v = (p.v[0]**2+p.v[1]**2)**0.5
    a = (p.a_g[0]**2+p.a_g[1]**2)**0.5
    dt = eta * fp.min(1./np.fabs(v),1./fabs(a))
    
    return dt

In [9]:
def SetPlanet(p,i):
    AU_in_km = 1.495979e8
    
    #circular velocity
    v_c = 0.0
    v_e = 0.0
    
    #Mercury
    if(i==0):
        p.a = 57909227.0 / AU_in_km
        p.e = 0.20563593 / AU_in_km
    #Venus    
    elif(i==1):
        p.a = 108209475.0 / AU_in_km
        p.e = 0.00677672 / AU_in_km
    #Earth
    elif(i==2):
        p.a = 1.0 / AU_in_km
        p.e = 0.01671123 / AU_in_km  
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    #get equiv circular velocity
    v_c = solar_circular_velocity(p, solar_system)
    #velocity at perihelion
    v_e = v_c*(1+p.e)**0.5
    #set velocity
    p.v[0] = 0.0 #no x velocity at perihelion
    p.v[1] = v_e #y velocity at perihelion (ccw)
    #calc grav accel from Sun
    solar_gravitational_acceleration(p)
    #set timestep
    p.dt = calc_dt(p)

In [11]:
def x_first_step(x_i, v_i, a_i, dt):
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [12]:
def v_full_step(x_i, v_i, a_ipoh, dt):
    return v_i + a_ipoh*dt;

In [13]:
def x_full_step(x_ipoh, v_ip1, a_ipoh, dt):
    return x_ipoh + v_ip1*dt